In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
import re
import logging
import os
from pathlib import Path
from functools import partial
from itertools import chain
from uuid import uuid4

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots

import bhutils
from bhutils import bhutilspy as bh
# from api.ezutils import bh, mkt_defs, ccy_cals, mx_cals, mtg_dts, gens

from gioutils.utils import parse_offset, today, get_next_n, get_bday
from api.data.base import data_path, radar_path
from gioutils.gui.base import View, store, WidgetView, Tabs
from api.gui.params import curves, bbg_params, spot_params, fwd_params
from gioutils.blpw import BlpQuery
from gioutils.utils import RepeatedTimer

from gioutils.ezutils import bh, bhs, SQL, ZCILSHelper, get_price_info, get_id
from api.gui.theme import bg_color, grid_style
from api.gui.pandas import DFOutput, negative_red, custom_styler_row_line, DataGridGio

pd.options.plotting.backend = "plotly"

In [3]:
from api.gui.collectors import DataCollector, SpotForward, Futures, Market, Mtgs, FwdMatrix, VolMatrix
from api.gui.fly_grids import FlyGrid
from api.gui.editable_grid import EditableGrid
from api.gui.price_grids import PricingGrid
from api.gui.tables import SpotTable, FutureTable, GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable
from api.gui.grids import SpotGrid, FutureGrid, MtgGrid, MatGrid, GroupMktGrid # GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable
from api.gui.global_config import GlobalConfig
from api.gui.monitors import MktMonitor, VolMonitor

In [4]:
bq = BlpQuery(timeout=50000).start()

In [5]:
ccys = [
    'EUR', 
    'USD', 
    'GBP'
]

In [6]:
curves_df = pd.DataFrame(curves).set_index('ccy').loc[ccys]

In [7]:
fly_grid = FlyGrid()

In [8]:
pricer_grid = PricingGrid()

In [9]:
mm = MktMonitor()
vm = VolMonitor()

In [10]:
dc = DataCollector(
    curves=curves_df.melt()['value'].drop_duplicates().dropna().to_list(),
    bq=bq,
    collectors={
        'fg': fly_grid,
        'pg': pricer_grid,
        'sf': SpotForward(spot_params=spot_params, fwd_params=fwd_params),
        'fut': Futures(),
        'mkt': Market(),
        'mtgs': Mtgs(),
        'mat': FwdMatrix(),
        'vol_mat': VolMatrix(),
    },
    renderers={
        'mm': mm,
        'vm': vm
    }
)

In [11]:
gc = GlobalConfig(dc=dc)

In [12]:
tabs = Tabs(
    {
        'radar': v.Container(
            fluid=True,
            children=[
                v.Row(children=[v.Col(children=[gc.view])]),
                v.Row(children=[v.Col(children=[mm.view])])
            ],
        ),
        'vol':v.Container(
            fluid=True,
            children=[vm.view]
        ),
        'pricer': v.Container(
            fluid=True,
            children=[pricer_grid.view]
        ),
        'flies': v.Container(
            fluid=True,
            children=[
                fly_grid.fly_view,
                # v.Row(children=[v.Col(children=[fly_grid.view])])
            ],
        ),
        'watchlist': fly_grid.expr_dg.view,
    },
    force_grid_render=True,
    # tabs_kwargs={'class_': "mb-2"}
)
tabs

Card(children=[Tabs(children=[Tab(children=['radar']), Tab(children=['vol']), Tab(children=['pricer']), Tab(ch…

In [13]:
'ILM3NAVG Index',
'OB30C Index',

'GSUSFCI Index',

'BLMFCI Index',

'.MSFCI Index',


('.MSFCI Index',)

In [14]:
from gioutils.utils import get_xy

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils.validation  import check_is_fitted, NotFittedError

In [16]:
X_names = [
    'GT2 Govt',
    'GT10 Govt',
    'ES1 Index',
    'CDX HY CDSI GEN 5Y SPRD Curncy',
    'CDX IG CDSI GEN 5Y SPRD Curncy',
    'CL1 Comdty',
]

y_name = 'NFCIINDX Index'

In [26]:
def get_bbg_data(securities, date, bq, options=None):
    data = bq.bdh(
        securities=securities,
        fields='px_last',
        start_date=date,
        options=options
    )

    data_mat = data.pivot(index='date', columns='security', values='px_last')
    return data_mat

In [53]:


class ECOPred():
    def __init__(self, X_names, y_name, date=None, freq='W-Fri', model=LinearRegression()):
        self.X_names = X_names
        self.y_name = y_name
        self.date = date or pd.Timestamp(2013, 1, 1)
        self.model = model
        self.freq = freq
        self.fit_data = None
        self.last_y_date = None
        self.predict_data = None
        
    def get_bbg_data(self, X_names=None, y_name=None, date=None, bq=None, options=False):
        X_names = X_names if X_names is not None else self.X_names
        y_name = y_name if y_name is not None else self.y_name
        if isinstance(y_name, str):
            y_name = [y_name]
        date = date or self.date
        data  = get_bbg_data(securities=[*X_names, *y_name], date=date, bq=bq, options=options)
        return data
        
    def get_reg_data(self, data_mat):
        X_names = self.X_names
        y_name = self.y_name
        y_ = data_mat[y_name].dropna()
        self.y_ = y_
        data_ = data_mat
        if freq := self.freq:
            data_ = data_mat.resample(self.freq).last()
        data_ = data_.ffill().dropna().loc[:y_.index[-1]]
        X, y = get_xy(data_[X_names], data_[y_name], diff=True)
        return X, y
        
    def fit(self, bq=None, **kwargs):
        data_mat = self.get_bbg_data(bq=bq)
        self.fit_data = data_mat
        X, y = self.get_reg_data(data_mat)
        self.X_reg = X
        self.y_reg = y
        self.model.fit(X, y)
        
    def get_predict_data(self, bq):
        if not self.last_y_date:
            y_date = self.fit_data[self.y_name].dropna().index[-1]
        else:
            y_date = self.last_y_date
        
        
        y_data = self.get_bbg_data(X_names=[], date=y_date, bq=bq, options=False)
        X_data = self.get_bbg_data(y_name=[], date=y_date, bq=bq, options=None)
        self.last_y_date = y_data.dropna().index[-1]
        self.pred_data = pd.concat([X_data, y_data], axis=1)
        return self.pred_data
    
    def get_last_yh(self, bq, **kwargs):
        try:
            check_is_fitted(self.model)
        except NotFittedError:
            self.fit(bq=bq)
        
        pred_data = self.get_predict_data(bq=bq)
        y_reg_hat = self.model.predict((pred_data.iloc[-1] - pred_data.iloc[0])[self.X_names].to_frame().T)[0]
        self.y_reg_hat = y_reg_hat
        self.last_y = pred_data[self.y_name].dropna()[0] 
        yh = y_reg_hat + self.last_y
        self.last_yh = yh
        return yh
        
    def get_y_reg_hat(self):
        return pd.Series(self.model.predict(self.X_reg), index=self.y_reg.index, name='yhat')
    
    def get_y_hat(self):
        yh = self.get_y_reg_hat()
        return (self.y_.shift() + yh).rename('yhat')
        
    def get_data(self, bq=None, cob_date=None, **kwargs):
        yh = self.get_last_yh(bq=bq, **kwargs)
        
        
        if cob_date:
            cob = self.fit_data[self.y_name].ffill().loc[get_bday(cob_date)]
        else:
            cob = self.last_y
            
        data = {
            'live': yh,
            'cob': cob,
            'change': yh - cob
        }
        
        self.data = pd.Series(data)
        return self.data

In [60]:
self.fit_data[self.y_name].ffill().loc[cob_date]

date
2013-01-01         NaN
2013-01-02         NaN
2013-01-03         NaN
2013-01-04   -0.566172
2013-01-07   -0.566172
                ...   
2022-11-15   -0.195138
2022-11-16   -0.195138
2022-11-17   -0.195138
2022-11-18   -0.195138
2022-11-21   -0.195138
Name: NFCIINDX Index, Length: 2580, dtype: float64

In [54]:
self = ECOPred(X_names=X_names, y_name=y_name)

In [55]:
self.get_data(bq=bq)

live     -0.188498
cob      -0.195138
change    0.006640
dtype: float64

In [56]:
self.get_data(bq=bq)

live     -0.188528
cob      -0.195138
change    0.006610
dtype: float64